In [1]:
# coding=utf-8

In [2]:
# imports libraries of python
from PIL import Image
import pytesseract
from pytesseract import image_to_string
import requests
from io import BytesIO
import sys
import re
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import time
from getpass import getpass

# Defining a function to check the validity of Xpath in the browser
def hasXpath(xpath):
    try:
        if wd.find_element(By.XPATH, xpath):
            return True
    except:
        return False


# Defining a function to check the validity of Xpath in the Post
def posthasXpath(xpath,post):
    try:
        if post.find_element(By.XPATH, xpath):
            return True
    except:
        return False

    
# Defining a function to check if facebook is logged in
def is_logged_in(driver):
    try:
        profile_svg_element = driver.find_element(By.XPATH, './/div[@aria-label="Account Controls and Settings"]')
        return True
    except NoSuchElementException:
        return False

    
# Defining a function to login
def login():
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    global wd
    wd = webdriver.Chrome(options=chrome_options)


    # Taking to the log-in page
    wd.get('https://www.facebook.com')

    # username and password
    usr = input('Enter username:\n')  # Username
    pas = getpass('Enter Password:\n')  # password

    username = wd.find_element(By.ID, 'email')
    username.send_keys(usr)
    sleep(0.5)

    # Locate password form by ID
    passw = wd.find_element(By.ID, 'pass')
    passw.send_keys(pas)
    sleep(0.5)

    # locate submit button by_xpath
    sign_in_button = wd.find_element(By.XPATH, '//*[@type="submit"]')
    sign_in_button.click()
    sleep(3)
    
    if is_logged_in(wd):
        print("logged in")
    else:
        print('log_in Error')

        
# Defining a function for initializing the search for a given keyword
def initialize_search(keyword):
    print("----------------------------xxxx----------------------------")
    print(f'Searching for {keyword}')

    # Open the file in write mode with the keyword provided
    global f 
    f = open(keyword + "_facebook.txt", "w", encoding="utf-8")
    
    head = f"'post_no', 'post_type', 'is_text_only' ,'user_name', 'post_time','likes','num_coments','num_shares','post_content','Cmt_content','post_url'\n"
    f.write(head)
    
    # Entering search page for facebook with the keyword
    wd.get('https://www.facebook.com/search/posts?q=' + keyword)

    

def scroll_to_bottom(wd, num_scrolls=3):
    j = 0
    while j < num_scrolls:
        # Scroll down to bottom so as to load more content
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)
        j = j + 1
    sleep(5)
    

def extract_user_name(name,post_type):
    user_name = None
    if post_type != "reel":
        user_name = name.text
        if not user_name:
            Element_HTML = name.get_attribute("outerHTML")
            user_name = Element_HTML.strip("</span>")
    
    elif post_type == "reel":
        Element_HTML = name.get_attribute("outerHTML")
        match = re.search(r'target="_blank">(.*?)</span>', Element_HTML)
        if match:
            user_name= match.group(1)
        elif not user_name:
            match = re.search(r'target="_blank">(.*?)</a>', Element_HTML)
            if match:
                user_name= match.group(1)
    return user_name


def extract_post_content(post):
    post_texts = post.find_elements(By.XPATH, post_content_xpath)
    post_content = ""
    for post_text in post_texts:
        content = post_text.text
        if not content:
            Element_HTML = post_text.get_attribute("outerHTML")
            match = re.search(r'text-align: start;">(.*?)<', Element_HTML)
            if match:
                content = match.group(1)
            else:
                match = re.search(r'text-align: start;">(.*?)<', Element_HTML)
                if match:
                    content = match.group(1)
                else:
                    match = re.search(r'tabindex="0">(.*?)<', Element_HTML)
                    if match:
                        content = match.group(1)
                    else:
                        content= Element_HTML
            post_content = post_content + content + " "
    return post_content
    

def extract_reels_info(post):
    #post_time for reels
    reel_date = post.find_element(By.XPATH, reel_date_xpath)
    Element_HTML = reel_date.get_attribute("outerHTML")
    post_time = re.search(r'x1qrby5j">(.*?)</span>', Element_HTML).group(1)

    #link for reels
    if (posthasXpath(reels_link_xpath, post)):
        allp = post.find_element(By.XPATH, reels_link_xpath)
        post_url = allp.get_attribute("href")

    other_content = post.find_elements(By.XPATH, './/span[@class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1j85h84"]')
    content_text = ""
    for content in other_content:
        content_line = content.text
        if not content_line:
            content_line = content.get_attribute("outerHTML")
        content_text = content_text + content_line + "\n"
        
    return post_time, post_url, content_text


def extract_interaction_data(post):
    likes, num_comments, num_shares = "","",""
    #No of likes for post
    if posthasXpath(likes_num_xpath, post):
        No_likes = post.find_element(By.XPATH, likes_num_xpath)
        likes = No_likes.text
        if not likes:
            Element_HTML = No_likes.get_attribute("outerHTML")
            match = re.search(r'<span class="x1e558r4">(.*?)</span>', Element_HTML)
            if match:
                likes = match.group(1)

    #no. of comments and shares                
    if posthasXpath(comment_num_xpath, post) and post_type != "reel":
        No_comments = post.find_elements(By.XPATH, comment_num_xpath)
        if len(No_comments)>1:
            num_comments= No_comments[1].text 
            if not num_comments:
                Element_HTML = No_comments[1].get_attribute("outerHTML")
                match = re.search(r'dir="auto">(.*?)</span>', Element_HTML)
                if match:
                    num_comments = match.group(1)

        if len(No_comments)>2:
            num_shares= No_comments[2].text 
            if not num_shares:
                Element_HTML = No_comments[2].get_attribute("outerHTML")
                match = re.search(r'dir="auto">(.*?)</span>', Element_HTML)
                if match:
                    num_shares = match.group(1)
        
    return likes, num_comments, num_shares



In [3]:
        
# main 
try:
    if is_logged_in(wd):
        print("Already logged in")
        
except:
    login()
    
keyword = ["stockmarket", "shear-market", "nifty", "banknifty", "investment", "intraday", "options trading"]

Enter username:
arushinde789@gmail.com
Enter Password:
········
logged in


In [4]:
fb_feed_xpath = '//div[@class="x193iq5w x1xwk8fm"]'
fb_artical_xpath = '//div[@class="x1a2a7pz"]'
general_u_name_xpath = './/span[@class="xt0psk2"]/a/strong/span'
group_post_u_name_xpath = './/span[@class="xt0psk2"]/a/span'
text_post_xpath = './/div[@class="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs"]'
reels_u_name_xpath = './/span[@class="x65f84u"]//a'
reels_link_xpath = './/div[@class="x1n2onr6"]/div[1]/div/a[@role = "link"]'
reel_date_xpath = './/span[@class="x4k7w5x x1h91t0o x1h9r5lt x1jfb8zj xv2umb2 x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1qrby5j"]'
post_content_xpath =  './/div[@class="xu06os2 x1ok221b"]/span/div/div'
likes_num_xpath = './/span[@class="x1e558r4"]'
comment_num_xpath = './/div[@class="x1n2onr6"]/div/div[2]/div'

for i in range(0, len(keyword)):
    initialize_search(keyword[i])
    NUM_SCROLLS = 10                         # change with no. of scrolls
    scroll_to_bottom(wd,NUM_SCROLLS)   
    sleep(2)

# --------------------------------xxxxxx----------------------------------------

    if hasXpath(fb_feed_xpath):   #xpath for facebbok feed:        
        posts = wd.find_elements(By.XPATH, fb_artical_xpath)   #xpath for facebbok artical:
        print(f'has {len(posts)}  post')
        sleep(5)
        for post_no, post in enumerate(posts):
            print('\n\n----------xx----------')
            print(f'\nlooking inside post no: {post_no} ')
            
            post_type, is_text_only = "Unknown" , 0
            user_name,post_time,likes,num_comments,num_shares,post_content,Cmt_content,post_url = "","","","","","","",""

            # Taking the user name from the post with the Xpath provided
            if posthasXpath(general_u_name_xpath, post):
                post_type = "general"
                name = post.find_element(By.XPATH, general_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                likes, num_comments, num_shares = extract_interaction_data(post)
            
            if posthasXpath(group_post_u_name_xpath, post):
                post_type = "group_post"
                name = post.find_element(By.XPATH, group_post_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                likes, num_comments, num_shares = extract_interaction_data(post)
            
            if posthasXpath(reels_u_name_xpath, post):
                post_type = "reel"
                name = post.find_element(By.XPATH, reels_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                post_time, post_url, post_content = extract_reels_info(post)            
            
            # post_content
            if posthasXpath(post_content_xpath, post):
                post_content = extract_post_content(post)
                print(post_content)
            
            if posthasXpath(text_post_xpath, post):
                is_text_only = 1
                post_text = post.find_element(By.XPATH, text_post_xpath)
                post_content = post_text.text
                if not post_content:
                    Element_HTML = post_text.get_attribute("outerHTML")
                    match = re.search(r'">(.*?)</div>', Element_HTML)
                    if match:
                        post_content= match.group(1)
#                 print(post_content)
                
                
                    

            # links and post_times for other post
            if posthasXpath('.//a[@class = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm" and @role = "link" and @tabindex="0"]', post):
                all_a_s = post.find_elements(By.TAG_NAME, 'a')    # Find all a elements 
                post_url = all_a_s[-3].get_attribute("href")
#                 print("url: ", post_url)
                total_a = len(all_a_s)
                if total_a>4:
                    a_s= all_a_s[4].get_attribute("outerHTML")
                    svg_id = re.search(r'<use xlink:href="#SvgT(\d)" xmlns:xlink="http://www.w3.org/1999/xlink"></use>', a_s)
                    if svg_id:
                        svg = int(svg_id.group(1))
                        date = wd.find_elements(By.ID, f"SvgT{svg}")
                        Element_HTML = date[0].get_attribute("outerHTML")
                        post_time = re.search(r'visible;">(.*?)</text>', Element_HTML)
                        if post_time:
                            post_time= post_time.group(1)
                            print("Post Time : ", post_time)
                    if not svg_id:
                        a_s= all_a_s[3].get_attribute("outerHTML")
                        svg_id = re.search(r'<use xlink:href="#SvgT(\d)" xmlns:xlink="http://www.w3.org/1999/xlink"></use>', a_s)
                        if svg_id:
                            svg = int(svg_id.group(1))
                            date = wd.find_elements(By.ID, f"SvgT{svg}")
                            Element_HTML = date[0].get_attribute("outerHTML")
                            post_time = re.search(r'# visible;">(.*?)</text>', Element_HTML)
                            if post_time:
                                post_time= post_time.group(1)
                                print("Post Time : ", post_time)


        
            print('opening url')
            if post_url:
                try:# Open a new tab
                    wd.execute_script("window.open();")
                    # Switch to the newly opened tab
                    new_tab = wd.window_handles[-1]
                    wd.switch_to.window(new_tab)
                    wd.get(post_url)
                    print("new link loaded")
                    sleep(5)
                    try:
                        button_element = wd.find_element(By.XPATH, './/div[@class = "x6s0dn4 x78zum5 xdj266r x11i5rnm xat24cr x1mh8g0r xe0p6wg"]/div[@role= "button"]')
                        button_element.click()
                        sleep(0.5)
                    except:
                        print("it had error 1") 
                    try:
                        button_element = wd.find_element(By.XPATH, './/div[@role = "menu" and @class = "x1n2onr6 xcxhlts x1fayt1i"]//div[@class = "x78zum5 xdt5ytf x1iyjqo2 x1n2onr6"]/div/div[3]')
                        button_element.click()   
                        print("sucses")
                        sleep(3)
                    except:
                        print("it had error 2")
                    try:
                        button_element = wd.find_element(By.XPATH, './/span[@class = "x78zum5 x1iyjqo2 x21xpn4 x1n2onr6"]/div[@role ="button"]')
                        button_element.click()                                       
                        print("sucses")
                        sleep(5)
                    except:
                        print("it had error 3")


                    if hasXpath('//div[@class="x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz" and @role="article"]'):
                        comments = wd.find_elements(By.XPATH, '//div[@class="x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz" and @role="article"]')
                        print(f"has {len(comments)} comments")
                        Cmt_content = ""
                        for x,y in enumerate( comments):
                            try:
                                commenter_name = y.find_element(By.XPATH, './/span[@class = "x3nfvp2"]/span' )
                                cmt_user_name= commenter_name.text
                            except:
                                cmt_user_name = "is_top_fan"
                            if posthasXpath('.//div[@class= "x1lliihq xjkvuk6 x1iorvi4"]/span/div/div',y):
                                comment_lines = y.find_elements(By.XPATH, './/div[@class= "x1lliihq xjkvuk6 x1iorvi4"]/span/div/div' )
                                com_lines = ""
                                for line in comment_lines:
                                    line_text = line.text
                                    com_lines = com_lines+line_text
                            Cmt_content= Cmt_content+f" comment_{x+1}, {cmt_user_name} : {com_lines} | "
#                         print(Cmt_content)
                
                except KeyboardInterrupt:
                    print("Keyboard Interrupt")
                    sleep(0.5)
                        
                finally:
                    # Close the new tab
                    wd.close()
                    # Switch back to the original tab
                    original_tab = wd.window_handles[0]
                    wd.switch_to.window(original_tab)


                
            to_write = f"'post_{post_no}:', '{post_type}','{is_text_only}' ,'{user_name}', '{post_time}','{likes}','{num_comments}','{num_shares}' ,'{post_content}','{Cmt_content}','{post_url}'\n"
            print(f"'post_{post_no}:',\nPost_type:\t\t'{post_type}',\nis_text_only:\t\t{is_text_only},\nUserName:\t\t'{user_name}',\nPost_time:\t\t'{post_time}',\nNo_likes:\t\t'{likes}',\nnumber_comments:\t'{num_comments}',\nnum_shares:\t\t'{num_shares}'  ,\nPost_content:\t'{post_content}',\nComments:\t'{Cmt_content}',\n{post_url}\n\n")
            cleaned_text = to_write.replace("\n", "  ").replace("\t", "  ")
            f.write(f'{cleaned_text} \n')
            
            

----------------------------xxxx----------------------------
Searching for stockmarket
has 23  post


----------xx----------

looking inside post no: 0 
Stock market mei girawat mei kya kare ? Join 25th jan stock market class.. republic day discount call 9708094321 
opening url
new link loaded
it had error 1
it had error 2
it had error 3
has 2 comments
'post_0:',
Post_type:		'general',
is_text_only:		0,
UserName:		'Dr. Mukul Agrawal',
Post_time:		'',
No_likes:		'64',
number_comments:	'3',
num_shares:		'2'  ,
Post_content:	'Stock market mei girawat mei kya kare ? Join 25th jan stock market class.. republic day discount call 9708094321 ',
Comments:	' comment_1, Abhishek Singh : No idea |  comment_2, Veer Sinha : Omg | ',
https://www.facebook.com/themukulagrawal?__cft__[0]=AZUagTCS99trYk6iOVGeqm-E74Lz2yPn7hU6yuJy1OE14qnbRN_-A7CarEWEXoY_MZjchKhOjiigvSL4y-ikwgbrJmgzspvg0mjO-XJ_CX0NZKLsGdS6ltADZjPvV9gITKXUIlIu_Ex7pCuks9R1REgaIZtxEjKHYYnRym4Aatb_iEnpXHD46raKyPp2VA8gDULcYEaouPgxQpO0QQeTJZGE&__

opening url
new link loaded
it had error 1
it had error 2
it had error 3
'post_8:',
Post_type:		'reel',
is_text_only:		0,
UserName:		'Zee Business',
Post_time:		'Nov 9',
No_likes:		'',
number_comments:	'',
num_shares:		''  ,
Post_content:	'<span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1j85h84"><span><span class="xh99ass"><span><span class="xzpqnlu xjm9jq1 x6ikm8r x10wlt62 x10l6tqk x1i1rx1s">&nbsp;</span><span aria-hidden="true"> · </span></span></span>Reels<span class="xh99ass"><span><span class="xzpqnlu xjm9jq1 x6ikm8r x10wlt62 x10l6tqk x1i1rx1s">&nbsp;</span><span aria-hidden="true"> · </span></span></span><span class="x4k7w5x x1h91t0o x1h9r5lt x1jfb8zj xv2umb2 x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1qrby5j">Nov 9</span><span class="xh99ass"><span><span class="xzpqnlu xjm9jq1 x6ikm8r x10wlt62 x10l6tqk x1i1rx1s">&nbsp;</span><span aria-hidden="true"> · </span></span></span><div class="x3nfvp2 x1q0g3np"><div class="x150jy0e"><spa

opening url
new link loaded
it had error 1
it had error 2
it had error 3
'post_16:',
Post_type:		'reel',
is_text_only:		0,
UserName:		'Ankur Upadhyay',
Post_time:		'Jan 17',
No_likes:		'',
number_comments:	'',
num_shares:		''  ,
Post_content:	'Reels
  · Jan 17
  ·
2.5K
48
19
<span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1j85h84">Hz47TzyMb.com</span>
',
Comments:	'',
https://www.facebook.com/reel/1453855602203871/?s=single_unit&__cft__[0]=AZV58p2A0bQZwOS7M163cFkTTXdbMhz-c1flvjonqR7oeuLlTskeRHGDitxZ18XXrSqbRQn0lAukHuAnBSeTZVB3IDEUXH1DMa3A9Q1RGFNHKTOBJVVMjDi03QWp7lKykP-hfNsSHX_qqj2rJkHAPt2RvKTvs6yCrPZi51yflYcApQ&__tn__=H-R




----------xx----------

looking inside post no: 17 
opening url
new link loaded
it had error 1
it had error 2
it had error 3
'post_17:',
Post_type:		'reel',
is_text_only:		0,
UserName:		'Money Mantra with Deepak Rohilla',
Post_time:		'Jan 18',
No_likes:		'',
number_comments:	'',
num_shares:		''  ,
Post_content:	'Reels
  · Jan 18
  ·
Give!
<span cl

Post Time :  15h
opening url
new link loaded
it had error 1
it had error 2
it had error 3
has 2 comments
'post_2:',
Post_type:		'group_post',
is_text_only:		1,
UserName:		'Delhi NCR Connect',
Post_time:		'15h',
No_likes:		'13',
number_comments:	'55',
num_shares:		''  ,
Post_content:	'Share market trader hi yaha par',
Comments:	' comment_1, Ankit Tiwari : Yes |  comment_2, Kumar Praveen : Paisa lgao or 6 percent per month pao or paisa safe pura capital | ',
https://www.facebook.com/groups/345164958966979/?hoisted_section_header_type=recently_seen&multi_permalinks=2708118322671619&__cft__[0]=AZWEq04uvQQxPwBP2rlrnA4Rjzhac33Rb9r2qCOINV9Xdrdgeckp_XgFHvPM40-QiJVM06I8KYloAR4pKEwYwkjgFxGh35-4aA-gd-eyS6XdhY_criLsOGroB6T7u7EtLY4roNqsccukG4u3lIrcYRcZA7oZIVyigqXObT5mlhcOIRaOrGo9_k-ZfnkjvdAtNRc&__tn__=-UC%2CP-R




----------xx----------

looking inside post no: 3 
Post Time :  January 10 at 9:29 PM
opening url
new link loaded
it had error 1
it had error 2
it had error 3
has 2 comments
'post_3:',
P

First we give a profit of one lakh rupees, then you have to give 30 percent as long as you wish, call watch and message on the given link  
opening url
new link loaded
sucses
it had error 3
has 4 comments
'post_10:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Share Market',
Post_time:		'',
No_likes:		'1',
number_comments:	'6',
num_shares:		''  ,
Post_content:	'First we give a profit of one lakh rupees, then you have to give 30 percent as long as you wish, call watch and message on the given link  ',
Comments:	' comment_1, Ideal Home : Agar paise Dube to kitna doge |  comment_2, Ideal Home : Agar paise Dube to kitna doge |  comment_3, Sachin Verma : agar pahele he dub Jaye to |  comment_4, Rahul Sharma : Any adviser call on 9157909742 | ',
https://www.facebook.com/photo/?fbid=122148235238033630&set=gm.6578990302206270&idorvanity=4000544730050853&__cft__[0]=AZWFAJCfggp9gpn8EYM27CNT72XGOHiwR-v5Fr0zmRS0Q1ziv1vLHrOXibnW7_r6oTRBMAlDfE-xhNjH5pJW-mS7VFqlRu5jPuEuJsAmLp0gRYY_5Z9WM5G

...

new link loaded
sucses
it had error 3
has 6 comments
'post_1:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Groww Discussion Group.',
Post_time:		'2d',
No_likes:		'42',
number_comments:	'18',
num_shares:		''  ,
Post_content:	'Rs.7.90 to Rs.50.00  profit booked after 3:00 Pm  ',
Comments:	' comment_1, Vishal Boston Vishal : Kab liya tha 7 rs bala |  comment_2, Manoj Haripal : Wah bhai bhai |  comment_3, Rishi Dahiya Rdzz : Bhai ab ye taqnik apnani pdegy |  comment_4, Arbind Lala Srivastav : Great Bhai... |  comment_5, Ranjeet Kumar Awasthi : Nirupom Ghosh |  comment_6, Ehsan Armani : Hii | ',
https://www.facebook.com/photo/?fbid=2058563004542922&set=gm.811847170763942&idorvanity=559012172714111&__cft__[0]=AZVJC_6kNsKUU6IRtjXn4BpgVV5schCVcwFICSvTbRAbOZIH8r87zrVnkK6lbLa2qH5Uw5Iqtf5vPYHhJrBzwL7OQJ7u9Pnci5nGoo0MlhbXYGSp7yVBC1nOwqYjI3jdJsOnxJw6Eii87pjDRYS_GuBqAdwbrT0LKEKIgqJFx1Qg9PWpjbc0Fcdozc9WZEtExp4&__tn__=EH-R




----------xx----------

looking inside post no: 2 
  
opening 

new link loaded
sucses
it had error 3
has 6 comments
'post_10:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Penny Stocks 2024',
Post_time:		'',
No_likes:		'4',
number_comments:	'11',
num_shares:		''  ,
Post_content:	' Whoever wants PDF book of technical analysis, please message. Free Free Free Free Book will be available. In telegram   ',
Comments:	' comment_1, Rambaran Maurya Rambaran : 3 |  comment_2, Mike Great : this is wonderful and beyond my expectations. I never knew that this little Invest I did will change my entire life I’m so happy today because of your guidance Liam George may God bless youLiam George |  comment_3, Yatin Patel : 8 |  comment_4, Sushil Raj : 3 |  comment_5, Mukhtar Umar : https://t.me/+DSC_YO1Fru9hMGRk |  comment_6, Zeeshan Ahmad : 3 | ',
https://www.facebook.com/photo/?fbid=341621678735252&set=gm.1088289062619060&idorvanity=836213937826575&__cft__[0]=AZUxe8RFSGYuU1-2b-VZmE8lcxt_0cMkgl0IZH35XzcoiPqn2QkNqZgrsJWEiexzvj_I944YOQjbCJwJSqHaqOypveXiSox

 Whoever wants PDF book of technical analysis, please message. Free Free Free Free Book will be available. In telegram   
opening url
new link loaded
sucses
it had error 3
has 6 comments
'post_18:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Penny Stocks 2024',
Post_time:		'',
No_likes:		'14',
number_comments:	'116',
num_shares:		''  ,
Post_content:	' Whoever wants PDF book of technical analysis, please message. Free Free Free Free Book will be available. In telegram   ',
Comments:	' comment_1, Mk Yadav Yadav : नानी |  comment_2, Suresh Shishpal Verma : Same |  comment_3, Kiran Nimbalkar : Nani |  comment_4, Hassan M Aji : https://t.me/+X_ivZlssqq03NDY0 |  comment_5, Priyansha Dubey : Ani |  comment_6, Irfan Qureshi : Nanai | ',
https://www.facebook.com/photo/?fbid=122154148406020841&set=gm.1087705392677427&idorvanity=836213937826575&__cft__[0]=AZXbsdvxKoQcw0Lmdvo9KXeFdqeXBVl3_GfdcGGyNEQ1o0eiSDz9vAmR_GpwX6p0diSfwoCqIuCgjtRQEtJOUyeXSc7OfX4rCySlNF-XWL-k1Ma4d2zDWiIkTs7taudNOv


opening url
new link loaded
it had error 1
it had error 2
it had error 3
'post_27:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Penny Stocks 2024',
Post_time:		'',
No_likes:		'6',
number_comments:	'',
num_shares:		''  ,
Post_content:	'',
Comments:	'',
https://www.facebook.com/photo/?fbid=228148183678207&set=gm.2694115927418277&idorvanity=2592581174238420&__cft__[0]=AZVGUFGndQpdtUxvHBuH0eMSz6X9mudMcTEKWqDHOy0SH0Y8cm8pNXIXZVMyvG1WjDlrIZWNzZv8UW1L3cepn3Hk60Go6z80JJCR-DgR_7CVnC3tK8Ns0181FBNcT3EmfRtvyq1LBsoGIRkFiqUUzuAaiWtk7SpilHbMOC51oGYDLhyhO66nlR2Rx7RPMFDg2Wc&__tn__=EH-R




----------xx----------

looking inside post no: 28 

opening url
new link loaded
sucses
it had error 3
has 6 comments
'post_28:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Stock Market Update',
Post_time:		'',
No_likes:		'11',
number_comments:	'50 comments',
num_shares:		''  ,
Post_content:	'',
Comments:	' comment_1, Rajesh Tandan : Yes |  comment_2, Advocates Pramit Agrahari : Ha |  comme


KeyboardInterrupt

